# Données factices

## Introduction


## Les classes d'hydratation, ou « seeders »

Laravel inclut une méthode simple pour « hydrater » votre base de données avec des données factices en utilisant des classes spécifiques appelées **seeders**. Toutes ces classes sont stockées dans le répertoire `database/seeds`. Les classes d'hydratation peuvent avoir le nom que vous souhaitez, mais elles devraient probablement suivre une convention raisonnable, comme `UserSeeder`, etc. A titre d'exemple, une classe `DatabaseSeeder` est définie pour vous. 

### Écriture de seeders

Pour créer un « _seeder_ », exécutez la commande `make:seeder`. Tous les seeders générés par le framework seront placés dans le répertoire `database/seeds` :

In [ ]:
%%bash

php artisan make:seeder ProductSeeder

Une classe de seeder ne contient qu'une seule méthode par défaut : `run`. Cette méthode est appelée lorsque la commande `db:seed` est exécutée. Dans la méthode run, vous pouvez insérer des données dans votre base de données comme vous le souhaitez. Vous pouvez utiliser le constructeur de requêtes pour insérer manuellement des données ou vous pouvez utiliser les usines de modèles Eloquent.

In [ ]:
%%php

public function run()
{
    DB::table('users')->insert([
        'name' => Str::random(10),
        'email' => Str::random(10).'@gmail.com',
        'password' => Hash::make('password'),
    ])
}

Dans cet exemple, on utilise la classe `DB`, qui permet d'écrire des requêtes SQL, pour créer un utilisateur assorti de données aléatoires.

Une classe d'hydratation peut en exécuter d'autres :

In [ ]:
%%php

public function run()
{
    $this->call([
        UserSeeder::class,
        PostSeeder::class,
        CommentSeeder::class,
    ]);
}

Pour exécuter la procédure, on utiliseune autre commande `db:seed` :

In [ ]:
%%bash

# Lancer tous les _seeders_
php artisan db:seed

#Lancer un _seeder_ particulier
php artisan db:seed --class=PostSeeder

## Utiliser des fabriques, ou « factories »

Même si les classes d'hydratation se suffisent à elles-mêmes, ce n'est généralement pas le moyen le plus pratique de créer des objets factices. On préfère déléguer cette tâche à une classe spécifique, que Laravel appelle des « factories ». Cette appellation est un peu trompeuse car elle fait écho au _design pattern_ qui porte le même nom.

Une fabrique est une classe qui contient une méthode principale `definition`. En voici un exemple, donné par Laravel :

In [ ]:
%%php

public function definition()
{
    return [
        'name' => $this->faker->name(),
        'email' => $this->faker->unique()->safeEmail(),
        'email_verified_at' => now(),
        'password' => '$2y$10$92IXUNpkjO0rOQ5byMi.Ye4oKoEa3Ro9llC/.og/at2.uheWG/igi', // password
        'remember_token' => Str::random(10),
    ];
}

Nous voyons que cette méthode décrit une procédur pour créer un objet (ici un `User`).

On remarque l'utilisation d'un objet `faker`,dont nous allons reparler par la suite.

Une fois cela fixé, la tâcehe de la classe d'hydratation est d'appeler les fabriques :

In [ ]:
%%php

public function run()
{
    Post::factory()
        ->count(25)
        ->create();

}

Nous voyons dans cet exemple que la création de multiples instances se fait simplement avec la méthode `count`. Ici, nous créons 25 objets.

Les fabriques disposent de deux méthodes pour créer des objets :
- `make` se contente de créer les objets
- `create` les enregistre en plus dans la base de données.

### Fabriques et états

En dhos de la mtéhode `definition`, les fabriques peuvent comporter de méthodes arbitraires pour engendrer certaines parties de l'objet. Par exemple, créons la méthode suivante, qui définit une entreprise et un mail pour un employé :

In [ ]:
%%php

public function professionalContact(): self
{
    return $this->state([
        'name' => $this->faker->unique()->company,
        'email' => $this->faker->email,
    ]);
}

Nous pouvons maintenant insérer cette méthode dans l'appel de la fabrique :

In [ ]:
%%php

public function run()
{
    Employee::factory()
        ->count(25)
        ->professionalContact()
        ->create();

}

Cette notion d'état (« _state_ ») est très pratique pour configurer une partie de l'objet, certaines propriétés ayant des rapprts entre elle (p.ex.  dans une entreprise, le poste occupé et le salaire). Cela va nous permettre d'engendrer des données aléatoires et néanmoins crédibles.

Par ailleurs, les fabriques ccepteraient aussi la sybntaxe suivante :

In [ ]:
%%php

Employee::factory([
    'name' => $this->faker->unique()->company,
    'email' => $this->faker->email,
])
    ->count(50)
    ->create();

mais on voit qu'elle nettement moins pratique et uniforme.

### Objets factices et associations

Généralement, les entités du modèle de données sont liées entre elles par des associations. Les fabriques savent créer ces associations automatiquement avec la méthode `each` :

In [ ]:
%%php

Employee::factory()
    ->count(50)
    ->create()
    -each(Task::Factory()->count(5)->create());

LA procédure peut être un peu plus compliquée si l'on souhaite réutiliser ces objets (p.ex. dans les associations Many-to-Many).

## FakerPHP

FakerPHP est un outil qui permet d'engendre des données « crédibles » et pas seulement des chaînes de caratères ou des nombres aléatoires.

La bibliothèque est intégrée par défaut à Laravel, ce qui fait que, comme nous l'avons vu plus haut,il suffit de l'appeler.

La seule configuration de FakerPHP est la langue. Dans une ceraine mesure, il est possible d'engendrer des valeurs dans les langues différentes (des textes, des nom propres, etc.). Cela n'est pas toujours très probant, les classes en dehors de l'anglais ne sont pas exhaustivement développées, mais cela peut être utile.

pour clas, il faut aller dans le fichier de configuration `/config/app.php` et changer la valeur :

In [ ]:
¨¨php

[
    // ...
    'faker_locale' => 'fr_FR',
    // ...
]

FakerPHP est une boîte à outils qui fonctionne à partir de « Formatter » pour es types d'informations très variées. 

Une fonctionnalité importante de FakerPHP est de pouvoir engendre des valeurs uniques avec la méthode `unique` :

%%php

$name = $this->faker->unique()->company;

Toutes les classes disponibles sont documentées sur le [site de la bibliothèque](https://fakerphp.github.io/)